In [2]:
import numpy as np 
import cv2
import os

In [3]:
def reverseColumn(img):
    return img[::-1 , :]
    


In [4]:
def reverseRow(img):
    return img[: ,::-1]

In [5]:
def getTranspose(img):
    return img.swapaxes(0, 1)

In [6]:
def reverseEntireMatrix(img):
    return img[::-1 , ::-1]

In [7]:
def RotateImageBy90(img):
    img = reverseColumn(img)
    img = getTranspose(img)
    return img

In [8]:
def RotateImageBy180(img):
    img = reverseRow(img)
    img = reverseEntireMatrix(img)

    return img

In [9]:
def RotateImageBy270(img):
    img = reverseRow(img)
    img = getTranspose(img)
    return img

In [10]:
def FlipImageHorizontally(img):
    img = reverseRow(img)
    return img

In [11]:
def FlipImageVertically(img):
    img =  reverseEntireMatrix(img)
    return img

In [12]:
def getImageSize(img):
    height, width = img.shape[:2]
    return height , width

In [13]:
def ScaleImage(img , newHeight , newWidth):
    img = cv2.resize(img, (newWidth, newHeight))
    return img

In [14]:
def IncreaseBrightness(img , value):
    img = img + value
    img = np.clip(img, 0, 255)
    return img


In [15]:
def DecreaseBrightness(img , value):
    img = img - value
    img = np.clip(img, 0, 255)
    return img

In [16]:
def ChangeContrast(img , alpha): # if alpha > 1 -> increase contrast , if alpha < 1 -> decrease contrast 
    img = alpha * img 
    img = np.clip(img , 0 , 255)
    return img

In [17]:
def saveImage(img , path , num):
    folder_name = os.path.basename(path)
    cv2.imwrite(f"{path}\\{folder_name}_augImage_{num}.jpg", img)

In [ ]:
def ApplyAugumentationTechniques(images , folderPath):
    n = 0 
    cnt = 0
    H, W, C = 256, 256, 3  
    augmented_image = np.zeros((H, W, C), dtype=np.uint8)
    for cnt in range(295):
        img = images[cnt % len(images)]
        match n:
            case 0:
              augmented_image = RotateImageBy90(img)  
              break
            case 1:
              augmented_image = RotateImageBy180(img) 
              ""
            case 2:
              augmented_image = RotateImageBy270(img) 
              break
            case 3:
              augmented_image = FlipImageHorizontally(img)
              break
            case 4:
              augmented_image = FlipImageVertically(img)
              break
            case 5:
              h, w = img.shape[:2]
              nh = np.random.randint(int(h*0.5), int(h*2.0))  
              nw = np.random.randint(int(w*0.5), int(w*2.0))
              augmented_image = ScaleImage(img , nh , nw)
              break
            case 6:
              value = np.random.randint(1 , 100)
              augmented_image = IncreaseBrightness(img , value)
              break
            case 7:
              value = np.random.randint(-100 , -1)
              augmented_image = DecreaseBrightness(img , value)
              break
            case 8:
              alpha = np.random.uniform(1.1, 2.5) 
              augmented_image = ChangeContrast(img, alpha)
              break
            case 9:
              alpha = np.random.rand()
              augmented_image = ChangeContrast(img , alpha)
              break
        n = (n + 1) % 10
        saveImage(augmented_image , folderPath , cnt) 
        print(f"augumented_image {cnt} saved successfuly!" )
        cnt += 1

In [19]:
folderPaths = [
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\glass",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\metal",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\paper",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\plastic",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\trash"
      ]
for folder in folderPaths:
    images = []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)  # <-- correct
        if os.path.isfile(file_path) and filename.lower().endswith('.jpg'):
            img = cv2.imread(file_path)
            if img is None:
                print("Failed to load:", file_path)
                continue
            images.append(img)
    ApplyAugumentationTechniques(images, folder)

Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\2ec9d19b-8027-4c77-a13f-5eee033b9868.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\31381a44-38d6-4a44-9384-7690727801bc.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\345bdb67-4190-4235-a16f-b60c1556a28d.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\38b3e4da-738c-4694-a946-55101b25ad53.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\4840d678-7af4-4a2d-bda1-338c2f2a59c5.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\509251d8-4e3a-4f1e-aabc-4d034b0f2455.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\5b7da318-c2ab-4c29-8ace-19895a890840.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\8617221e-dc90-48fe-a116-46350b5f814e.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\88ce5fbf-e9c7-40ad-87a6-deffe95d8ee8.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop

In [ ]:
import numpy as np 
import cv2
import os

In [ ]:
def reverseColumn(img):
    return img[::-1 , :]
    


In [ ]:
def reverseRow(img):
    return img[: ,::-1]

In [ ]:
def getTranspose(img):
    return img.swapaxes(0, 1)

In [ ]:
def reverseEntireMatrix(img):
    return img[::-1 , ::-1]

In [ ]:
def RotateImageBy90(img):
    img = reverseColumn(img)
    img = getTranspose(img)
    return img

In [ ]:
def RotateImageBy180(img):
    img = reverseRow(img)
    img = reverseEntireMatrix(img)

    return img

In [ ]:
def RotateImageBy270(img):
    img = reverseRow(img)
    img = getTranspose(img)
    return img

In [ ]:
def FlipImageHorizontally(img):
    img = reverseRow(img)
    return img

In [ ]:
def FlipImageVertically(img):
    img =  reverseEntireMatrix(img)
    return img

In [ ]:
def getImageSize(img):
    height, width = img.shape[:2]
    return height , width

In [ ]:
def ScaleImage(img , newHeight , newWidth):
    img = cv2.resize(img, (newWidth, newHeight))
    return img

In [ ]:
def IncreaseBrightness(img , value):
    img = img + value
    img = np.clip(img, 0, 255)
    return img


In [ ]:
def DecreaseBrightness(img , value):
    img = img - value
    img = np.clip(img, 0, 255)
    return img

In [ ]:
def ChangeContrast(img , alpha): # if alpha > 1 -> increase contrast , if alpha < 1 -> decrease contrast 
    img = alpha * img 
    img = np.clip(img , 0 , 255)
    return img

In [ ]:
def saveImage(img , path , num):
    folder_name = os.path.basename(path)
    cv2.imwrite(f"{path}\\{folder_name}_augImage_{num}.jpg", img)

In [ ]:
def ApplyAugumentationTechniques(images , folderPath):
    n = -1 
    cnt = 0
    H, W, C = 256, 256, 3  
    augmented_image = np.zeros((H, W, C), dtype=np.uint8)
    while cnt < 164:
      for img in images:
        if cnt >= 164:
          return
        cnt += 1
        n = (n + 1) % 10
        match n:
            case 0:
              augmented_image = RotateImageBy90(img)  
            case 1:
              augmented_image = RotateImageBy180(img) 
            case 2:
              augmented_image = RotateImageBy270(img) 
            case 3:
              augmented_image = FlipImageHorizontally(img)
            case 4:
              augmented_image = FlipImageVertically(img)
            case 5:
              h, w = img.shape[:2]
              nh = np.random.randint(int(h*0.5), int(h*2.0))  
              nw = np.random.randint(int(w*0.5), int(w*2.0))
              augmented_image = ScaleImage(img , nh , nw)
            case 6:
              value = np.random.randint(1 , 100)
              augmented_image = IncreaseBrightness(img , value)
            case 7:
              value = np.random.randint(1 , 100)
              augmented_image = DecreaseBrightness(img , value)
            case 8:
              alpha = np.random.uniform(1.1, 2.5) 
              augmented_image = ChangeContrast(img, alpha)
            case 9:
              alpha = np.random.rand()
              augmented_image = ChangeContrast(img , alpha)
        saveImage(augmented_image , folderPath , cnt) 
        print(f"augumented_image {cnt} saved successfuly!" )
        

In [ ]:
folderPaths = [
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\glass",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\metal",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\paper",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\plastic",
    r"C:\Users\Fatma Ibrahim\Desktop\Dataset\trash"   
      ]
for folder in folderPaths:
    images = []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder,filename)
        if os.path.isfile(file_path) and filename.lower().endswith('.jpg'):
          img = cv2.imread(file_path)
          if img is None:
            os.remove(file_path)
            continue 
          images.append(img)
   # ApplyAugumentationTechniques(images , folder)
    
            


Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\2ec9d19b-8027-4c77-a13f-5eee033b9868.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\31381a44-38d6-4a44-9384-7690727801bc.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\345bdb67-4190-4235-a16f-b60c1556a28d.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\38b3e4da-738c-4694-a946-55101b25ad53.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\4840d678-7af4-4a2d-bda1-338c2f2a59c5.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\509251d8-4e3a-4f1e-aabc-4d034b0f2455.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\5b7da318-c2ab-4c29-8ace-19895a890840.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\8617221e-dc90-48fe-a116-46350b5f814e.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard\88ce5fbf-e9c7-40ad-87a6-deffe95d8ee8.jpg
Failed to load: C:\Users\Fatma Ibrahim\Desktop

In [ ]:
img = images[0]
rotate90img = RotateImageBy90(img) 
rotate90img = cv2.cvtColor(rotate90img, cv2.COLOR_BGR2RGB)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\rotated90_image.jpg", rotate90img)
rotate180img = RotateImageBy180(img)
rotate180img = cv2.cvtColor(rotate180img, cv2.COLOR_BGR2RGB)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\rotated180_image.jpg", rotate180img)
rotate270img = RotateImageBy270(img)
rotate270img = cv2.cvtColor(rotate270img, cv2.COLOR_BGR2RGB)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\rotated270_image.jpg", rotate270img)

True

In [ ]:
flipHimg = FlipImageHorizontally(img)
flipVimg = FlipImageVertically(img)
flipHimg = cv2.cvtColor(flipHimg, cv2.COLOR_BGR2RGB)
flipVimg = cv2.cvtColor(flipVimg, cv2.COLOR_BGR2RGB)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\flippedH_image.jpg", flipHimg)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\flippesV_image.jpg", flipVimg)

True

In [ ]:
scaledImage = ScaleImage(img ,1000 , 1000)
scaledImage = cv2.cvtColor(scaledImage ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\scaled_image.jpg", scaledImage)

True

In [ ]:
incBrightness = IncreaseBrightness(img , 100)
incBrightness = cv2.cvtColor(incBrightness ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\plusprightnness_image.jpg", incBrightness)
decBrightness = DecreaseBrightness(img , 100)
decBrightness = cv2.cvtColor(decBrightness ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\minusprightnness_image.jpg", decBrightness)

True

In [ ]:
incContrast = ChangeContrast(img , 5)
incContrast = np.clip(incContrast, 0, 255).astype(np.uint8)
incContrast = cv2.cvtColor(incContrast ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\pluscontrast_image.jpg", incContrast)
deccontrast = ChangeContrast(img , 0.5)
deccontrast = np.clip(deccontrast, 0, 255).astype(np.uint8)
deccontrast = cv2.cvtColor(deccontrast ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\minuscontrast_image.jpg", deccontrast)

True

In [23]:
img = images[0]
rotate90img = RotateImageBy90(img) 
rotate90img = cv2.cvtColor(rotate90img, cv2.COLOR_BGR2RGB)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\rotated90_image.jpg", rotate90img)
rotate180img = RotateImageBy180(img)
rotate180img = cv2.cvtColor(rotate180img, cv2.COLOR_BGR2RGB)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\rotated180_image.jpg", rotate180img)
rotate270img = RotateImageBy270(img)
rotate270img = cv2.cvtColor(rotate270img, cv2.COLOR_BGR2RGB)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\rotated270_image.jpg", rotate270img)

True

In [27]:
flipHimg = FlipImageHorizontally(img)
flipVimg = FlipImageVertically(img)
flipHimg = cv2.cvtColor(flipHimg, cv2.COLOR_BGR2RGB)
flipVimg = cv2.cvtColor(flipVimg, cv2.COLOR_BGR2RGB)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\flippedH_image.jpg", flipHimg)
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\flippesV_image.jpg", flipVimg)

True

In [32]:
scaledImage = ScaleImage(img ,1000 , 1000)
scaledImage = cv2.cvtColor(scaledImage ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\scaled_image.jpg", scaledImage)

True

In [33]:
incBrightness = IncreaseBrightness(img , 100)
incBrightness = cv2.cvtColor(incBrightness ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\plusprightnness_image.jpg", incBrightness)
decBrightness = DecreaseBrightness(img , 100)
decBrightness = cv2.cvtColor(decBrightness ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\minusprightnness_image.jpg", decBrightness)

True

In [37]:
incContrast = ChangeContrast(img , 5)
incContrast = np.clip(incContrast, 0, 255).astype(np.uint8)
incContrast = cv2.cvtColor(incContrast ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\pluscontrast_image.jpg", incContrast)
deccontrast = ChangeContrast(img , 0.5)
deccontrast = np.clip(deccontrast, 0, 255).astype(np.uint8)
deccontrast = cv2.cvtColor(deccontrast ,cv2.COLOR_BGR2RGB )
cv2.imwrite("C:\\Users\\Fatma Ibrahim\\Downloads\\images\\minuscontrast_image.jpg", deccontrast)

True